# Parallel Computation (Timing)

This notebook measures the efficiency gains when using parallelization during simulation. For the description of how LightCurveLynx parallelizes computation, along with technical details, see the [parallelization notebook](https://lightcurvelynx.readthedocs.io/en/latest/notebooks/parallel_runs.html).

We run multiple simulations of [sncomso](https://sncosmo.readthedocs.io/en/stable/)'s SALT2 model with varying levels of parallelization.

## Input Data

For the simulation we use a realistic survey cadence and set of passbands for LSST. For the survey we pre-download the v5.1.1 simulated survey from [https://s3df.slac.stanford.edu/data/rubin/sim-data/](https://s3df.slac.stanford.edu/data/rubin/sim-data/). For filters we use the ones defined by the LSST preset. Both are stored in a local "data" directory.

In [ ]:
from lightcurvelynx.astro_utils.passbands import PassbandGroup
from lightcurvelynx.obstable.opsim import OpSim

opsim_file = "../data/opsim/baseline_v5.1.1_10yrs.db"
ops_data = OpSim.from_db(opsim_file)
print(f"Loaded OpSim data with {len(ops_data)} entries.")

t_min, t_max = ops_data.time_bounds()
print(f"Time bounds of OpSim data: {t_min} to {t_max}")

table_dir = "../data/passbands/LSST"
passband_group = PassbandGroup.from_preset(preset="LSST", table_dir=table_dir)
print(f"Loaded passband group with {len(passband_group)} passbands.")

## Model

For the object model, we use a SALT2 model (sncosmo's "salt2-h17) with a position that is randomly drawn from the coverage of the survey and a `t0` that is drawn uniformly from the survey's time coverage. The remaining parameters are set as constants with: `redshift`=0.1, `x0`=1.0e-6, `x1`=0.5, and `c`=0.2.

Since the sncosmo models are only defined for a limited time window around t0, we extrapolate with a linear decay to zero over 50 days.

In [ ]:
from lightcurvelynx.math_nodes.np_random import NumpyRandomFunc
from lightcurvelynx.math_nodes.ra_dec_sampler import ApproximateMOCSampler
from lightcurvelynx.models.sncosmo_models import SncosmoWrapperModel
from lightcurvelynx.utils.extrapolate import LinearDecay

ra_dec_sampler = ApproximateMOCSampler.from_obstable(ops_data, depth=6)

source = SncosmoWrapperModel(
    "salt2-h17",
    t0=NumpyRandomFunc("uniform", low=t_min, high=t_max),
    x0=1.0e-6,
    x1=0.5,
    c=0.2,
    ra=ra_dec_sampler.ra,
    dec=ra_dec_sampler.dec,
    redshift=0.01,
    node_label="source",
    time_extrapolation=LinearDecay(decay_width=50.0),
)

## Simulation

We perform a basic parallelization using the (default) `ProcessPoolExecutor`. Note that this approach will have significant overhead per worker as it copies all the state.

In [ ]:
import functools
import timeit
import numpy as np

from lightcurvelynx.simulate import simulate_lightcurves

num_samples_list = [10_000, 20_000, 30_000, 40_000, 50_000]
num_jobs_list = [1, 2, 3, 4]
num_simulations = 10

all_times = np.zeros((len(num_samples_list), len(num_jobs_list), num_simulations))
for sample_idx, num_samples in enumerate(num_samples_list):
    for job_idx, num_jobs in enumerate(num_jobs_list):
        print(f"Starting {num_simulations} simulations of {num_samples} samples with num_jobs={num_jobs}:")
        simulate_func = functools.partial(
            simulate_lightcurves,
            model=source,
            num_samples=num_samples,
            obstable=ops_data,
            passbands=passband_group,
            num_jobs=num_jobs,
            obs_time_window_offset=(-100, 400),
            progress_bar=False,  # Disable progress bar
        )

        for trial in range(num_simulations):
            print(f"  Simulation {trial + 1}/{num_simulations} with num_jobs={num_jobs}:")
            t = timeit.timeit(simulate_func, number=1)
            all_times[sample_idx, job_idx, trial] = t
            print(f"  Time for this simulation: {t:.2f} seconds")

        total_time = np.sum(all_times[sample_idx, job_idx])
        average_time = np.mean(all_times[sample_idx, job_idx])
        print(f"Time (sec) for num_jobs={num_jobs}: Total={total_time:.2f}. Average={average_time:.2f}")
        print("-" * 50)

print(all_times)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
for jobs_idx, num_jobs in enumerate(num_jobs_list):
    ave_times = np.mean(all_times[:, jobs_idx, :], axis=1)
    ax.plot(num_samples_list, ave_times, label=f"num_jobs={num_jobs}")

ax.set_xlabel("Number of Samples")
ax.set_ylabel("Average Time (seconds)")
ax.set_title("Parallel Simulation Timing")
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
for sample_idx, num_samples in enumerate(num_samples_list):
    ave_times = np.mean(all_times[sample_idx, :, :], axis=1)
    lwr_delta = ave_times - np.min(all_times[sample_idx, :, :], axis=1)
    upr_delta = np.max(all_times[sample_idx, :, :], axis=1) - ave_times
    plt.errorbar(
        num_jobs_list,
        ave_times,
        yerr=[lwr_delta, upr_delta],
        label=f"num_samples={num_samples}",
        capsize=5,
        marker="o",
    )

ax.set_xlabel("Number of Jobs")
ax.set_ylabel("Average Time (seconds)")
ax.set_title("Parallel Simulation Timing")
ax.legend()
plt.show()

fig.savefig("parallel_simulation_timing.pdf")